In [1]:
import geopandas
import json
import shapely
import shapely.geometry
import xarray
import rasterio
import rioxarray
import os
import fiona
import nasa_hls
import urllib.request as urlreq
import pandas as pd
import numpy as np
import requests
import xmltodict
import shutil
import datetime
import boto3


from shapely.ops import transform
from shapely.geometry import Point
from shapely.geometry import Polygon
from pystac_client import Client 
from collections import defaultdict
from glob import glob
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
from tqdm import tqdm
from netrc import netrc
from subprocess import Popen
from platform import system
from getpass import getpass
from rasterio.session import AWSSession
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

In [64]:
##### START OPTIONS #####
cloud_thres = 5

root_path = "/data/"
req_path = "/data/requirements/"
## file paths
# spath = root_path + f"CDL_HLS_dataframe{yoi[0]}"
# image_index_file = root_path + f"image_index{yoi[0]}"
chip_file =  req_path + "chip_bbox.geojson"
chip_csv = req_path + "chip_tracker.csv"
kml_file = req_path + 'sentinel_tile_grid.kml'
cdl_reclass_csv = req_path + "cdl_freq.csv"
# tile_tracker_csv = root_path + "tile_tracker.csv"

## folder paths
# hdf_dir = root_path + "hdf/"
chip_dir = root_path + 'chips/'
tile_dir = root_path + 'tiles/'
# chip_dir_binary = root_path + 'chips_binary/'
chip_dir_multi = chip_dir + 'chips_multi/'

chip_dir_filt = chip_dir + 'chips_filtered/'
# chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = chip_dir + 'chips_multi_filtered/'

chip_fmask_dir = chip_dir + 'chips_fmask/'

#####  END OPTIONS  #####

In [65]:
dirs_to_make = [tile_dir, chip_dir, chip_dir_multi, chip_fmask_dir]
for folder in dirs_to_make:
    try:
        os.makedirs(folder)
    except FileExistsError:
        # directory already exists
        print('pass')
        pass


pass
pass
pass
pass


In [66]:
with open(chip_file, "r") as file:
    chips = json.load(file)
    
chip_ids = []
chip_x = []
chip_y = []
for item in chips['features']:
    #print(item)
    chip_ids.append(item['properties']['id'])
    chip_x.append(item['properties']['center'][0])
    chip_y.append(item['properties']['center'][1])

In [67]:
with open("/data/requirements/chip_ids.json", "w") as f:
    json.dump(chip_ids, f, indent=2)

In [68]:
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [69]:
def find_tile(x,y):
# Identify closest tile
    s = (tile_x - x)**2+(tile_y - y)**2
    tname = tile_name[np.argmin(s)]
    return(tname)

In [70]:
chip_df = pd.DataFrame({"chip_id" : chip_ids, "chip_x" : chip_x, "chip_y" : chip_y})
chip_df['tile'] = chip_df.apply(lambda row : find_tile(row['chip_x'], row['chip_y']), axis = 1)
chip_df.tail(5)

,chip_id,chip_x,chip_y,tile
4995,chip_090_484,-84.446559,45.575077,16TFR
4996,chip_177_428,-89.751220,40.674821,16TBL
4997,chip_106_429,-89.268425,44.931751,16TCQ
4998,chip_312_160,-109.614541,31.902541,12SXA
4999,chip_198_312,-99.007068,39.532747,14SMJ


In [71]:
chip_df.to_csv(req_path + "chip_df.csv", index=False)

In [72]:
tiles = chip_df.tile.unique().tolist()
tiles[0:5]

['13TDE', '16SDD', '13SFV', '14TNS', '14UMU']

In [73]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

In [75]:
# Original 5 percentage query
tile_list = []
print(f"There are a total of {len(tiles)} tiles")
tile_iter = 0
for current_tile in tiles[0:1]:

    chip_df_filt = chip_df.loc[chip_df.tile == current_tile]#.reset_index()
    first_chip_id = chip_df_filt.chip_id.iloc[0]
    first_chip_index_in_json = chip_ids.index(first_chip_id)
    roi = chips['features'][first_chip_index_in_json]['geometry']

    search = catalog.search(
        collections = ['HLSS30.v2.0'],
        intersects = roi,
        datetime = '2022-03-01/2022-09-30',
    ) 
    
    num_results = search.matched()
    item_collection = search.get_all_items()
    
    tile_name = "T" + current_tile
    iter_items = 0
    for i in tqdm(item_collection ,desc=f"({tile_iter}/{len(tiles)})"):
        if i.id.split('.')[2] == tile_name:
            if i.properties['eo:cloud_cover'] <= cloud_thres:
                response = requests.get(i.assets['metadata'].href)
                if response.status_code == 200:
                    temp_xml = response.text
                    temp_xml = xmltodict.parse(temp_xml)
                    temp_dict = {"tile_id": tile_name, "cloud_cover": i.properties['eo:cloud_cover'],
                                 "date": datetime.datetime.strptime(i.properties['datetime'].split('T')[0], "%Y-%m-%d"), 
                                 "spatial_cover": int(temp_xml['Granule']['AdditionalAttributes']['AdditionalAttribute'][3]['Values']['Value']),
                                 "http_links": {"B02": i.assets['B02'].href, "B03": i.assets['B03'].href, "B04": i.assets['B04'].href,  "B8A": i.assets['B8A'].href,
                                                "B11": i.assets['B11'].href, "B12": i.assets['B12'].href, "Fmask": i.assets['Fmask']},
                                "s3_links": {"B02": i.assets['B02'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B03": i.assets['B03'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B04": i.assets['B04'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B8A": i.assets['B8A'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "B11": i.assets['B11'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "B12": i.assets['B12'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "Fmask": i.assets['Fmask'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/')}}
                    tile_list.append(temp_dict)
                    iter_items += 1
                else: 
                    assert False, f"Failed to fetch XML from {i.assets['metadata'].href}. Error code: {response.status_code}"
            
    tile_iter += 1
    #print(f"Information for tile {tile_name} is collected, a total of {iter_items} out of {num_results} tiles pass the filter ({tile_iter}/{len(tiles)})")

    
tile_df = pd.DataFrame(tile_list)

There are a total of 491 tiles


(0/491): 100%|██████████████████████████████████████████████████████████████████████████| 81/81 [00:05<00:00, 15.63it/s]


In [76]:
tile_df.to_csv(req_path + "tile_df.csv", index=False)

In [77]:
tile_df = pd.read_csv("/data/requirements/tile_df.csv")

In [78]:
tile_df

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,1,2022-03-14,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022073...
1,T13TDE,0,2022-04-08,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022098...
2,T13TDE,0,2022-04-18,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022108...
3,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
4,T13TDE,2,2022-05-26,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022146...
5,T13TDE,4,2022-06-15,100,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022166...
6,T13TDE,1,2022-06-27,26,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022178...
7,T13TDE,1,2022-07-02,26,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022183...
8,T13TDE,0,2022-07-17,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022198...
9,T13TDE,1,2022-08-06,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022218...


In [79]:
tile_df[tile_df.tile_id == "T13TDE"]

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,1,2022-03-14,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022073...
1,T13TDE,0,2022-04-08,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022098...
2,T13TDE,0,2022-04-18,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022108...
3,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
4,T13TDE,2,2022-05-26,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022146...
5,T13TDE,4,2022-06-15,100,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022166...
6,T13TDE,1,2022-06-27,26,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022178...
7,T13TDE,1,2022-07-02,26,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022183...
8,T13TDE,0,2022-07-17,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022198...
9,T13TDE,1,2022-08-06,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022218...


In [80]:
len(np.unique(tile_df.tile_id))

1

In [114]:
def spatial_filtering (dataframe):
    """
        Using spatial coverage percentage to filter chips

        Args:
            dataframe: A pandas dataframe that generated previously
    """
    cover_list = [100, 90, 80, 70, 60, 50]
    tile_list_ft = []
    tile_list = dataframe.tile_id.unique().tolist()
    
    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile]
        for cover_pct in cover_list:
            
            temp_df_filtered = temp_df[temp_df.spatial_cover >= cover_pct]
            if len(temp_df_filtered) >= 3:
                for i in range(len(temp_df_filtered)):
                    tile_list_ft.append(temp_df_filtered.iloc[i])
                break
    
    tile_df_filtered = pd.DataFrame(tile_list_ft)
    return tile_df_filtered

In [115]:
# check_spatial_cover(tile_df, 90)
cover_df = spatial_filtering(tile_df)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 216.27it/s]


In [118]:
def select_scenes(dataframe):
    select_tiles = []
    tile_list = dataframe.tile_id.unique().tolist()

    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile].sort_values('date').reset_index(drop=True)
        select_tiles.extend([temp_df.iloc[0], temp_df.iloc[len(temp_df) // 2], temp_df.iloc[-1]])

    return pd.DataFrame(select_tiles).reset_index(drop=True)

In [117]:
selected_tiles = select_scenes(cover_df)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 597.91it/s]


In [85]:
selected_tiles.head()

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
1,T13TDE,4,2022-06-15,100,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022166...
2,T13TDE,5,2022-09-23,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022266...


In [86]:
selected_tiles[selected_tiles.tile_id == "T13TDE"]

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
1,T13TDE,4,2022-06-15,100,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022166...
2,T13TDE,5,2022-09-23,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022266...


In [ ]:
# selected_tiles.iloc[0].s3_links['Fmask'].split("/")

In [129]:
test1 = selected_tiles.iloc[0].s3_links
test1

"{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B02.tif', 'B03': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B03.tif', 'B04': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B04.tif', 'B8A': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B8A.tif', 'B11': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B11.tif', 'B12': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.B12.tif', 'Fmask': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111T174859.v2.0/HLS.S30.T13TDE.2022111T174859.v2.0.Fmask.tif'}"

In [ ]:
# test1['B02']

In [132]:
selected_tiles.to_csv(req_path + "selected_tiles.csv", index=False)

In [136]:
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
prompts = ['Enter NASA Earthdata Login Username: ',
           'Enter NASA Earthdata Login Password: ']

# Determine the OS (Windows machines usually use an '_netrc' file)
netrc_name = "_netrc" if system()=="Windows" else ".netrc"

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser(f"~/{netrc_name}")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
    # Set restrictive permissions
    Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)

In [137]:
s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'

In [138]:
def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()

In [ ]:
# temp_creds_req = get_temp_creds()
#temp_creds_req                      # !!! BEWARE, removing the # on this line will print your temporary S3 credentials.

In [ ]:
session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [ ]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [134]:
def tile_download(table, boto3_session):
    bucket = boto3_session
    info_list = []
    bands = ["B02","B03","B04","B8A","B11","B12","Fmask"]
    accept_tiles = np.unique(table.tile_id)
    for tile in tqdm(accept_tiles):
        temp_tb = table[table.tile_id == tile]
        for i in range(3):
            bands_dict = temp_tb.iloc[i].s3_links
            for band in bands:
                temp_key = bands_dict[band].replace("s3:/", "")
                temp_sav_path = f"/data/tiles/{bands_dict[band].split('/')[2]}/{bands_dict[band].split('/')[3]}"
                os.makedirs(f"/data/tiles/{bands_dict[band].split('/')[2]}", exist_ok=True)
                if not Path(temp_sav_path).is_file():
                    boto3_session.resource('s3').Bucket('lp-prod-protected').download_file(Key = temp_key, Filename = temp_sav_path)
            temp_dict = {"tile":tile, "timestep":i, "date":temp_tb.iloc[i].date, "save_path":f"/data/tiles/{bands_dict[band].split('/')[2]}/", "filename":bands_dict["B02"].split('/')[3].replace(".B02.tif","")}
            info_list.append(temp_dict)
    return pd.DataFrame(info_list)

In [ ]:
track_df = tile_download(selected_tiles, session)

In [ ]:
track_df.to_csv(req_path + "track_df.csv", index=False)

In [ ]:
track_df[temp_df.tile == "T13TDE"]

#############################################

reproject selected cog to cdl crs

In [ ]:
def reproject_hls_to_cdl(scene_folder,
                         bands = ["B02", "B03", "B04", "B08A","B11","B12", "Fmask"],
                         cdl_file = "/data/2021_30m_cdls_clipped.tif"):
    
    """
    This function receives the path to a folder that contains all GeoTIFF files (for various bands)
    of a HLS scene, and reprojects those to the target CDL CRS and grid. 
    
    Assumptions:
    - scene_folder has a file structure like: ".../<scene_id>/<scene_id>.<band_id>.tiff
    - scene_folder should not have a "/" at the end
    
    Inputs:
    - scene_folder: is the path to the folder that contains HLS GeoTIFF files for all bands of HLS
    - bands: list of bands of HLS that should be reprojected (default is all bands)
    - cdl_file: contains the path to the clipped CDL GeoTIFF file
    
    """
    
    for band in bands:
        xds = xarray.open_rasterio(f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.tif")
        cdl = xarray.open_rasterio(cdl_file)
        xds_new = xds.rio.reproject_match(cdl, resampling = Resampling.bilinear)
        xds_new.rio.to_raster(raster_path = f"{scene_folder}/{scene_folder.split('/')[-1]}.{band}.5070.tif")

In [ ]:
for tile in tiles_to_reproject:
    selected_images = glob(tif_dir + '*')
    #print(selected_images)
    selected_images = [image for image in selected_images if image[19:24] == tile]
    print(selected_images)
        ## reproject to cdl
    for k in range(len(selected_images)):
        image_id = selected_images[k]
        print(image_id)
        reproject_hls_to_cdl(image_id)
    tile_tracker = pd.read_csv(tile_tracker_csv)
    tile_tracker.loc[tile_tracker.tile == tile , 'tif_reproject'] = True
    tile_tracker.to_csv(tile_tracker_csv, index=False)

chipping

In [ ]:
 def check_qa(qa_path, shape,  valid_qa = [0, 4, 32, 36, 64, 68, 96, 100, 128, 132, 160, 164, 192, 196, 224, 228]):
    
    """
    This function receives a path to a qa file, and a geometry. It clips the QA file to the geometry. 
    It returns the number of valid QA pixels in the geometry, and the clipped values.
    
    Assumptions: The valid_qa values are taken from Ben Mack's post:
    https://benmack.github.io/nasa_hls/build/html/tutorials/Working_with_HLS_datasets_and_nasa_hls.html
    
    Inputs:
    - qa_path: full path to reprojected QA tif file
    - shape: 'geometry' property of single polygon feature read by fiona
    - valid_qa: list of integer values that are 'valid' for QA band.
    

    
    """
    with rasterio.open(qa_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
      #  print(out_image.shape)
        vals = out_image.flatten()
        unique, counts = np.unique(vals, return_counts=True)
        qa_df = pd.DataFrame({"qa_val" : unique, "counts" : counts})
        qa_df
        qa_df[~ qa_df.qa_val.isin(valid_qa)].sort_values(['counts'], ascending = False)
        qa_df['pct'] = (100 *qa_df['counts'])/(224.0 * 224.0)
        
        bad_qa = qa_df[~ qa_df.qa_val.isin(valid_qa)].sort_values(['counts'], ascending = False)
        if len(bad_qa) > 0:
            highest_invalid_percent = bad_qa.pct.tolist()[0]
        else: 
            highest_invalid_percent = 0
        #ncell = len(vals)
        valid_count = sum(x in valid_qa for x in vals)
        return(valid_count, highest_invalid_percent, out_image[0])

In [ ]:
## set up CDL reclass
cdl_class_df = pd.read_csv(cdl_reclass_csv)
crop_dict = dict(zip(cdl_class_df.CDL_val, cdl_class_df.new_class_value))

In [ ]:
def crop_reclass(x):
    ## binary reclass
    crop_classes = [1,2,3,4,5,6,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,66,67,68,69,70,71,72,74,75,76,77,92,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,236,237,238,240,241,242,243,244,245,246,247,248,249,250,254]
    return(crop_classes.count(x))

c_rcl = np.vectorize(crop_reclass)


def crop_multi(x):
    return(crop_dict[x])

c_multi = np.vectorize(crop_multi)

In [ ]:
def process_chip(chip_id, 
                 chip_tile,
                 shape,
                 track_csv,
                 bands = ["B02", "B03", "B04", "B8A", "B11", "B12"]):
    
    """
    This function receives a chip id, HLS tile, chip geometry, and a list of bands to process. 
    
    Assumptions:
    
    Inputs:
    - chip_id: string of chip id, e.g. '000_001'
    - chip_tile: string of HLS tile , e.g. '15ABC'
    - shape: 'geometry' property of single polygon feature read by fiona
    
    The function writes out a multi-date TIF containing the bands for each of the three image dates for an HLS tile. 
    The function writes out a multi-date TIF containing the QA bands of each date.
    The function writes out a chipped version of CDL. 
    The function calls check_qa(), which makes assumptions about what QA pixels are valid.
    The function returns the number of valid QA pixels at each date, as a tuple.
    
    """
    ## get reprojected image paths
    tile_info_df = track_csv[track_csv.tile == chip_tile]
    
    # selected_image_folders = sorted(glob(f'/data/tif/*T{chip_tile}*'))
    selected_image_folders = tile_info_df.save_path.to_list()
   # print(selected_image_folders)
    
    assert len(selected_image_folders) == 3
    
    # first_image_date = selected_image_folders[0][25:32]
    # second_image_date = selected_image_folders[1][25:32]
    # third_image_date = selected_image_folders[2][25:32]
                     
    first_image_date = tile_info_df.iloc[0].date
    second_image_date = tile_info_df.iloc[1].date
    third_image_date = tile_info_df.iloc[2].date
    
    # first_date_images = sorted(glob(selected_image_folders[0] + '/*.5070.tif')) 
    # first_date_qa = [x for x in first_date_images if '.QA.' in x][0]
    # first_date_images.remove(first_date_qa)

    # second_date_images = sorted(glob(selected_image_folders[1] + '/*.5070.tif'))
    # second_date_qa = [x for x in second_date_images if '.QA.' in x][0]
    # second_date_images.remove(second_date_qa)
    
    # third_date_images = sorted(glob(selected_image_folders[2] + '/*.5070.tif'))
    # third_date_qa = [x for x in third_date_images if '.QA.' in x][0]
    # third_date_images.remove(third_date_qa)
    # all_date_images = first_date_images + second_date_images + third_date_images

    all_date_images = []
    all_date_qa = []
                     
    for i in range(3):
        for band in bands:
            all_date_images.append(tile_info_df.iloc[i].save_path + f"{tile_info_df.iloc[i].filename}.{band}.tif")
        all_date_qa.append(tile_info_df.iloc[i].save_path + f"{tile_info_df.iloc[i].filename}.Fmask.tif")
    
    # print(all_date_images)
    # print(len(all_date_images))


    # valid_first, bad_pct_first, qa_first = check_qa(first_date_qa, shape)
    # valid_second, bad_pct_second, qa_second = check_qa(second_date_qa, shape)
    # valid_third, bad_pct_third, qa_third = check_qa(third_date_qa, shape)
    print(all_date_qa[0:2])
    valid_first, bad_pct_first, qa_first = check_qa(all_date_qa[0], shape)
    valid_second, bad_pct_second, qa_second = check_qa(all_date_qa[1], shape)
    valid_third, bad_pct_third, qa_third = check_qa(all_date_qa[2], shape)
    
    qa_bands = []
    qa_bands.append(qa_first)
    qa_bands.append(qa_second)
    qa_bands.append(qa_third)
    qa_bands = np.array(qa_bands).astype(np.int16)
    
    # print(qa_bands.shape)
    # print(first_date_qa)
    assert len(all_date_images) == 3 * len(bands)
    
    out_bands = []
    
    for img in all_date_images:
        with rasterio.open(img) as src:
            out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
            out_meta = src.meta
            out_bands.append(out_image[0])
    
    out_bands = np.array(out_bands)
    # print(out_bands.shape)
    # print(out_image.shape)

    out_meta.update({"driver": "GTiff",
                     "height": out_bands.shape[1],
                     "width": out_bands.shape[2],
                     "count": out_bands.shape[0],
                     "transform": out_transform})
    
    # get NA count for HLS
    na_count = sum(out_bands.flatten() == -1000)
    
    # reclass negative HLS values to 0
    out_bands = np.clip(out_bands, 0, None)
    
    
    
    # write HLS chip to 'chips'
    with rasterio.open(chip_dir + "chip_" + str(chip_id) + "_merged.tif", "w", **out_meta) as dest:
        dest.write(out_bands)
    # # write HLS chip to 'chips_binary'
    # with rasterio.open(chip_dir_binary + "chip_" + str(chip_id) + "_merged.tif", "w", **out_meta) as dest:
    #     dest.write(out_bands)
    # # write HLS chip to 'chips_multi'
    # with rasterio.open(chip_dir_multi + "chip_" + str(chip_id) + "_merged.tif", "w", **out_meta) as dest:
    #     dest.write(out_bands)
      
    ## write QA bands
    out_meta.update({"driver": "GTiff",
                     "height": qa_bands.shape[1],
                     "width": qa_bands.shape[2],
                     "count": qa_bands.shape[0],
                     "transform": out_transform})
    
    with rasterio.open(chip_qa_dir + "chip_" + str(chip_id) + "_Fmask.tif", "w", **out_meta) as dest:
        dest.write(qa_bands)  
    
        
    # ## clip cdl to chip
    # with rasterio.open("/data/2021_30m_cdls_clipped.tif") as src:
    #     out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
    #     out_meta = src.meta
    #     colormap = src.colormap(1)

    # out_meta.update({"driver": "GTiff",
    #                  "height": out_image.shape[1],
    #                  "width": out_image.shape[2],
    #                  "transform": out_transform})
    # # write CDL chip to 'chips'
    # with rasterio.open(chip_dir + "chip_" + str(chip_id) + ".mask.tif", "w", **out_meta) as dest:
    #     dest.write(out_image)
    #     dest.write_colormap(1, colormap)
        
        
    # # write binary  reclassed CDL chip to chips_binary
    # out_image_binary = c_rcl(out_image).astype(np.uint8)
    # with rasterio.open(chip_dir_binary + "chip_" + str(chip_id) + ".mask.tif", "w", **out_meta) as dest:
    #     dest.write(out_image_binary)
    #     dest.write_colormap(1, colormap)
        
    # # write multiclass  reclassed CDL chip to chips_multi
    # out_image_multi = c_multi(out_image).astype(np.uint8)
    # with rasterio.open(chip_dir_multi + "chip_" + str(chip_id) + ".mask.tif", "w", **out_meta) as dest:
    #     dest.write(out_image_multi)
    #     dest.write_colormap(1, colormap)
    
    
    return(valid_first,
           valid_second,
           valid_third, 
           bad_pct_first,
           bad_pct_second,
           bad_pct_third,
           qa_first,
           qa_second,
           qa_third,
           na_count,
           first_image_date,
           second_image_date,
           third_image_date)

In [ ]:
tiles_to_chip = temp_df.tile.tolist()

In [ ]:
## process chips

for tile in tiles_to_chip:
    print(tile)
    chips_to_process = chip_df[chip_df.tile == tile.replace("T", "")].reset_index(drop = True)
    for k in range(len(chips_to_process)):
        current_id = chips_to_process.chip_id[k]
        chip_tile = chips_to_process.tile[k]
        # print(current_id)
        chip_index = chip_ids.index(current_id)

        chip_feature = chips['features'][chip_index]

        shape = [chip_feature['geometry']]

        ## do we want to scale/clip reflectances?
        full_tile_name = "T" + chip_tile
        valid_first,  valid_second, valid_third, bad_pct_first, bad_pct_second, bad_pct_third, qa_first, qa_second, qa_third, na_count, first_image_date, second_image_date, third_image_date = process_chip(current_id, full_tile_name, shape, track_df)

        chip_df_index = chip_df.index[chip_df['chip_id'] == current_id].tolist()[0]
        chip_df.at[chip_df_index, 'valid_first'] = valid_first
        chip_df.at[chip_df_index, 'valid_second'] = valid_second
        chip_df.at[chip_df_index, 'valid_third'] = valid_third
        chip_df.at[chip_df_index, 'bad_pct_first'] = bad_pct_first
        chip_df.at[chip_df_index, 'bad_pct_second'] = bad_pct_second
        chip_df.at[chip_df_index, 'bad_pct_third'] = bad_pct_third
        chip_df.at[chip_df_index, 'first_image_date'] = first_image_date
        chip_df.at[chip_df_index, 'second_image_date'] = second_image_date
        chip_df.at[chip_df_index, 'third_image_date'] = third_image_date
        chip_df['bad_pct_max'] = chip_df[['bad_pct_first', 'bad_pct_second', 'bad_pct_third']].max(axis=1)
        chip_df.at[chip_df_index, 'na_count'] = na_count
    # tile_tracker = pd.read_csv(tile_tracker_csv)
    # tile_tracker.loc[tile_tracker.tile == tile , 'chip'] = True
    # tile_tracker.to_csv(tile_tracker_csv, index=False)
chip_df.to_csv(chip_csv, index=False)

filter chips

In [ ]:
chip_df = pd.read_csv(chip_csv)

for tile in tiles_to_filter:
    print(tile)
    filtered_chips = chip_df[(chip_df.tile == tile) & (chip_df.bad_pct_max < 5) & (chip_df.na_count == 0)].chip_id.tolist()
    print(len(filtered_chips))
    for chip_id in filtered_chips:
        chip_files = glob('/data/chips/*' + chip_id + '*')
        for file in chip_files:
            name = file.split('/')[-1]
            shutil.copyfile(file, '/data/chips_filtered/' + name)
        chip_files_b = glob('/data/chips_binary/*' + chip_id + '*')
        for file in chip_files_b:
            name = file.split('/')[-1]
            shutil.copyfile(file, '/data/chips_binary_filtered/' + name)
        chip_files_multi = glob('/data/chips_multi/*' + chip_id + '*')
        for file in chip_files_multi:
            name = file.split('/')[-1]
            shutil.copyfile(file, '/data/chips_multi_filtered/' + name)
    
    tile_tracker = pd.read_csv(tile_tracker_csv)
    tile_tracker.loc[tile_tracker.tile == tile , 'filter_chips'] = True
    tile_tracker.to_csv(tile_tracker_csv, index=False)